In [ ]:
import hyperspy.api as hs
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import pandas as pd

from helper_functions import elementlines, nearestlines, theoretical_energy

In [ ]:
import os
path = "../../Masteroppgave/2023-03-08_EDS-Apreo/exports/"
files = os.listdir(path)
files

In [ ]:
s_list = [hs.load(path + '/' + file, signal_type='EDS_SEM') for file in files]

In [ ]:
zero_peak_slice = 33
scale = 0.01
offset = 0.1242 - zero_peak_slice * scale
channels = 2048
x = np.arange(offset, offset + channels * scale, scale)

In [ ]:
def figg(log=True):
    fig = go.Figure()
    fig.update_layout(margin=dict(l=5, r=5, t=5, b=5))
    fig.update_layout(xaxis_title='Energy (keV)', yaxis_title='Counts')
    if log:
        fig.update_yaxes(type="log")
    return fig


def add_all_element_lines(fig, element):
    lines = list(hs.material.elements[element].Atomic_properties.Xray_lines.as_dictionary().keys())
    for line in lines:
        line = element + '_' + line
        fig.add_vline(x=theoretical_energy(line), line_width=1, line_dash="dash", line_color="black", annotation_text=line, annotation_position="top right", annotation_textangle=90)

In [ ]:
# plot raw data

fig = figg(log=False)
for s in s_list:
    fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename))


add_all_element_lines(fig, 'Ga')
add_all_element_lines(fig, 'O')
add_all_element_lines(fig, 'C')
add_all_element_lines(fig, 'N')
add_all_element_lines(fig, 'Sb')
add_all_element_lines(fig, 'Si')

fig.show()

In [ ]:
# make a logaritmic plot
fig = figg()
for s in s_list:
    fig.add_trace(go.Scatter(x=x, y=s.data, name=s.metadata.General.original_filename))

fig.show()

In [ ]:
# showing the effect of different processTime

processTimeFiles = [
# 'GaSb_30kV_50pA_noPPUC.emsa', #  'PT 6 (no PPUC)',
'GaSb_30kV_50pA_processTime1.emsa',
'GaSb_30kV_50pA_processTime2.emsa',
'GaSb_30kV_50pA_processTime4.emsa',
'GaSb_30kV_50pA.emsa']

names = ['Process time 1', 'Process time 2', 'Process time 4', 'Process time 6']

processTimeData = [hs.load(path + '/' + file, signal_type='EDS_SEM').data for file in processTimeFiles]

fig = figg()
for i, s in enumerate(processTimeData):
    fig.add_trace(go.Scatter(x=x, y=s, name=names[i]))

fig.update_xaxes(range=[0.17, 1.3])
fig.update_yaxes(range=[2.2, 4.2])
fig.update_layout(title_text='Effect of different process times')
fig.update_layout(margin=dict(l=5, r=5, t=50, b=5))
fig.update_layout(legend=dict( yanchor="top", y=0.95, xanchor="left", x=0.02))
fig.update_layout(width=1000, height=600)


fig.show()

fig.write_html("figures/processTime.html")

In [ ]:
nearestlines(0.73)#.sort_values('energy [keV]')